# Import necessary packages

In [13]:
# Import all required packages
#import linearmodels
from panel_reg import PanelReg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial.distance as sd
import statsmodels.formula.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
from scipy.spatial.distance import euclidean, pdist, squareform
from fastdtw import fastdtw
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from numpy import genfromtxt
from datetime import datetime, timedelta
from getpass import getuser as get_user
from getpass import getpass as get_password
from os import environ as enviroment_vars
from os import setpgrp as set_process_group
from random import random
from shlex import split as cmd_split
from subprocess import Popen, PIPE, STDOUT
import pyodbc
import calendar
from sklearn.preprocessing import OneHotEncoder
import math
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

# Create Impala connection function 

In [14]:
def impala_select(cnxn,query,show=True,sort_col=None):
    data = pd.read_sql(query,cnxn)
    if show:
        if sort_col:
            print(data.sort_values(sort_col))
        else:
            print(data)
    return data

# Create Kerberos ticket function

In [15]:
##################################################
#
#    name:   authenticate_kerberos
#    desc:   function that checks if user has a non-expired Kerberos Ticket.
#            If ticket does not exist, or it is expired, will ask 
#            for password to generate a new Kerberos Ticket. 
#            Optionally can pass "force=True" to generate a new Ticket even if existing is not expired.
#            Optionally can pass "keytab=True" to use keytab for authentication instead of typing password.
#
#            *** You should include this at beginning of every script that connects to the Hadoop Cluster! ***
#
#    args:
#            force  - (Optional) Force re-creation of Kerberos Ticket.
#            keytab - (Optional) Use Keytab for authentication instead of password.
#                                This assumes that the keytab file is in the user's  
#                                home directory with the name: <username>.keytab.
#                          
#    returns:
#            boolean - True if ticket successfully gotten, false otherwise 
#
#################################################
def authenticate_kerberos(force=False,keytab=False):
    # get username
    username = get_user()
    # get kerberos realm
    krb_realm = "MILLER.LOCAL"
    
    # create 'klist" command
    cmd = "klist"
    # create linux command process
    process = Popen(cmd_split(cmd),stdout=PIPE,stdin=PIPE, stderr=STDOUT,preexec_fn=set_process_group)
    # wait until done, get result
    result = process.communicate(None)[0].decode("utf-8") 
    
    # check that we don't already have a valid ticket
    found_username = False
    expire_date = None
    for line in result.split("\n"):
        if username in line:
            found_username = True
            # update realm if different
            krb_realm = line.split("@")[-1]
        line = line.split()
        if len(line) == 5:
            try:
                expire_date = datetime.strptime(line[2] + " " + line[3],'%m/%d/%Y %H:%M:%S')
            except (ValueError) as e:
                pass
    
    if (found_username) and (expire_date != None) and (expire_date > datetime.today()) and (not force):
        return True
    else:
        # create 'kdestory" command
        cmd = "kdestroy"
        # create linux command process
        process = Popen(cmd_split(cmd),stdout=PIPE,stdin=PIPE, stderr=STDOUT,preexec_fn=set_process_group)
        # wait until done, get result
        result = process.communicate(None)[0].decode("utf-8") 

        if keytab:
            # create "kinit" command
            cmd = "kinit -kt "+ enviroment_vars["HOME"] + "/" + username + ".keytab " + username + "@" + krb_realm
            # create linux command process
            process = Popen(cmd_split(cmd),stdout=PIPE,stdin=PIPE, stderr=STDOUT,preexec_fn=set_process_group)
            # wait until done, get result
            result = process.communicate(None)[0].decode("utf-8").strip()
            # if result
            if result.strip():
                print("Failed to obtained Kerberos Ticket via Keytab.")
                # result
                print(result)
                return False
            # ticket got successfully
            else:
                print("Successfully obtained Kerberos Ticket via Keytab.\n")
                return True
        else:
            # create "kinit" command
            cmd = "kinit " + username + "@" + krb_realm
            # create linux command process
            process = Popen(cmd_split(cmd),stdout=PIPE,stdin=PIPE, stderr=STDOUT,preexec_fn=set_process_group)
            # wait until done, get result
            result = process.communicate(get_password("Password for " + username + "@" + krb_realm + ": ").encode('utf-8'))[0].decode("utf-8").strip()
        
            # if result
            if result.strip() != "Password for " + username + "@" + krb_realm + ":":
                print("Failed to obtained Kerberos Ticket via Password.")
                # result
                print(result)
                return False
            # ticket got successfully
            else:
                print("Successfully obtained Kerberos Ticket via Password.\n")
                return True


# Create Kerberos Ticket (input your password when prompted)

In [16]:
### Run Kerberos access functions, then run Hadoop query to get data. ###
success = authenticate_kerberos(force=False)

# Connect to Impala

In [17]:
# connect to Impala
cnxn = pyodbc.connect('DSN=Impala', autocommit=True)
cursor = cnxn.cursor()

# Set dynamic variables for SQL pull from Impala

In [18]:
#Most recent Saturday end date that you want to start the analysis from
weekendingdate_start = '06/20/20'

#Number of weeks you want to include in your model
num_weeks = 104

#Name of your personal dabase for creating temp tables
your_db = 'josiah_johnson'

#Nielsen parameters
channel = '%grocery%'
dma_names  = ['los angeles ca']
# state = ''
# planning_unit = ''
dma_names = "', '".join(dma_names)

#Dependent Segment
subsegment_names =['premium light','premium regular']
subsegment_names = "', '".join(subsegment_names)

#Kroger parameters
planning_units = ['pu - rbg ca','pu - northern ca']
planning_units = "', '".join(planning_units)

#create your case when statements for re-naming brpx values
brpx_renaming = """case
when brpx='oldmilwaukee_12_12' then 'econ_12_12'
when brpx='olympia_12_12' then 'econ_12_12'
when brpx='natural_12_12' then 'econ_12_12'
when brpx='natural_12_15' then 'econ_12_15'
when brpx='keystone_12_15' then 'econ_12_15'
when brpx='natural_12_18' then 'econ_12_18'
when brpx='hamms_12_24' then 'econ_12_24'
when brpx='natural_12_30' then 'econ_12_30'
when brpx='keystone_12_30' then 'econ_12_30'
when brpx='hamms_16_6' then 'econ_16_6'
when brpx='strongbow_11.2_6' then 'cider_12_6'
when brpx='strongbow_11.2_12' then 'cider_12_12'
when brpx='angryorchard_12_6' then 'cider_12_6'
when brpx='ace_12_6' then 'cider_12_6'
when brpx='stellaartois_12_6' then 'cider_12_6'
when brpx='2towns_12_6' then 'cider_12_6'
when brpx='wyders_12_6' then 'cider_12_6'
when brpx='crispin_12_6' then 'cider_12_6'
when brpx='space_12_6' then 'cider_12_6'
when brpx='smith&forge_12_6' then 'cider_12_6'
when brpx='commoncider_12_6' then 'cider_12_6'
when brpx='anthem_12_6' then 'cider_12_6'
when brpx='angryorchard_12_12' then 'cider_12_12'
when brpx='ace_12_12' then 'cider_12_12'
when brpx='crispin_12_12' then 'cider_12_12'
when brpx='2towns_12_12' then 'cider_12_12'
when brpx='stellaartois_12_12' then 'cider_12_12'
when brpx='angryorchard_12_24' then 'cider_12_24'
when brpx='townepark_12_6' then 'locraft_12_6'
when brpx='hebrew_12_6' then 'locraft_12_6'
when brpx='absolution_12_6' then 'locraft_12_6'
when brpx='buffalobills_12_6' then 'locraft_12_6'
when brpx='housebeer_12_6' then 'locraft_12_6'
when brpx='leinenkugels_12_6' then 'natcraft_12_6'
when brpx='shocktop_12_6' then 'natcraft_12_6'
when brpx='omission_12_6' then 'locraft_12_6'
when brpx='widmerbrothers_12_6' then 'locraft_12_6'
when brpx='fourcorners_12_6' then 'locraft_12_6'
when brpx='chihuahua_12_6' then 'locraft_12_6'
when brpx='shiner_12_6' then 'natcraft_12_6'
when brpx='bjs_12_6' then 'locraft_12_6'
when brpx='gooseisland_12_6' then 'natcraft_12_6'
when brpx='redhook_12_6' then 'natcraft_12_6'
when brpx='muchoaloha_12_6' then 'locraft_12_6'
when brpx='northcoast_12_6' then 'locraft_12_6'
when brpx='oskarblues_12_6' then 'locraft_12_6'
when brpx='magichat_12_6' then 'locraft_12_6'
when brpx='trumer_12_6' then 'locraft_12_6'
when brpx='firestone_12_6' then 'locraft_12_6'
when brpx='figueroamountain_12_6' then 'locraft_12_6'
when brpx='angelcity_12_6' then 'locraft_12_6'
when brpx='kona_12_6' then 'natcraft_12_6'
when brpx='bluemoon_12_6' then 'natcraft_12_6'
when brpx='lostcoast_12_6' then 'locraft_12_6'
when brpx='hopvalley_12_6' then 'locraft_12_6'
when brpx='goldenroad_12_6' then 'locraft_12_6'
when brpx='wiens_12_6' then 'locraft_12_6'
when brpx='abita_12_6' then 'locraft_12_6'
when brpx='samueladams_12_6' then 'natcraft_12_6'
when brpx='saintarcher_12_6' then 'locraft_12_6'
when brpx='anchor_12_6' then 'locraft_12_6'
when brpx='bigskybrewing_12_6' then 'locraft_12_6'
when brpx='roughdraft_12_6' then 'locraft_12_6'
when brpx='garage_12_6' then 'locraft_12_6'
when brpx='10barrel_12_6' then 'locraft_12_6'
when brpx='fourpeaks_12_6' then 'locraft_12_6'
when brpx='ninkasi_12_6' then 'locraft_12_6'
when brpx='alaskan_12_6' then 'locraft_12_6'
when brpx='lagunitas_12_6' then 'natcraft_12_6'
when brpx='hangar24_12_6' then 'locraft_12_6'
when brpx='21stamendment_12_6' then 'locraft_12_6'
when brpx='sierranevada_12_6' then 'natcraft_12_6'
when brpx='speakeasy_12_6' then 'locraft_12_6'
when brpx='boochcraft_12_6' then 'locraft_12_6'
when brpx='ritual_12_6' then 'locraft_12_6'
when brpx='mikehess_12_6' then 'locraft_12_6'
when brpx='newbelgium_12_6' then 'natcraft_12_6'
when brpx='bootlegger_12_6' then 'locraft_12_6'
when brpx='andersonvalley_12_6' then 'locraft_12_6'
when brpx='founders_12_6' then 'natcraft_12_6'
when brpx='karlstrauss_12_6' then 'locraft_12_6'
when brpx='deschutes_12_6' then 'natcraft_12_6'
when brpx='lakefront_12_6' then 'locraft_12_6'
when brpx='telegraph_12_6' then 'locraft_12_6'
when brpx='odell_12_6' then 'locraft_12_6'
when brpx='jamaicabrand_12_6' then 'locraft_12_6'
when brpx='brooklyn_12_6' then 'locraft_12_6'
when brpx='athletic_12_6' then 'locraft_12_6'
when brpx='fall_12_6' then 'locraft_12_6'
when brpx='mojave_12_6' then 'locraft_12_6'
when brpx='lobotomy_12_6' then 'locraft_12_6'
when brpx='dogfishhead_12_6' then 'locraft_12_6'
when brpx='motherearthbrewing_12_6' then 'locraft_12_6'
when brpx='bearrepublic_12_6' then 'locraft_12_6'
when brpx='strand_12_6' then 'locraft_12_6'
when brpx='avery_12_6' then 'locraft_12_6'
when brpx='cigarcity_12_6' then 'locraft_12_6'
when brpx='threeweav3rs_12_6' then 'locraft_12_6'
when brpx='drakes_12_6' then 'locraft_12_6'
when brpx='coronado_12_6' then 'locraft_12_6'
when brpx='motherearthbrewco_12_6' then 'locraft_12_6'
when brpx='duckfoot_12_6' then 'locraft_12_6'
when brpx='sufferfest_12_6' then 'locraft_12_6'
when brpx='uinta_12_6' then 'locraft_12_6'
when brpx='boulevard_12_6' then 'locraft_12_6'
when brpx='mission_12_6' then 'locraft_12_6'
when brpx='dustbowl_12_6' then 'locraft_12_6'
when brpx='fremont_12_6' then 'locraft_12_6'
when brpx='santamonica_12_6' then 'locraft_12_6'
when brpx='rogue_12_6' then 'locraft_12_6'
when brpx='blackmarket_12_6' then 'topcraft_12_6'
when brpx='fortpoint_12_6' then 'topcraft_12_6'
when brpx='slobrew_12_6' then 'topcraft_12_6'
when brpx='latitude33_12_6' then 'topcraft_12_6'
when brpx='greatdivide_12_6' then 'topcraft_12_6'
when brpx='smogcity_12_6' then 'topcraft_12_6'
when brpx='lefthand_12_6' then 'topcraft_12_6'
when brpx='breckenridge_12_6' then 'topcraft_12_6'
when brpx='thorn_12_6' then 'topcraft_12_6'
when brpx='bells_12_6' then 'topcraft_12_6'
when brpx='eelriver_12_6' then 'topcraft_12_6'
when brpx='turaalcoholickombucha_12_6' then 'topcraft_12_6'
when brpx='kneedeep_12_6' then 'topcraft_12_6'
when brpx='secondchance_12_6' then 'topcraft_12_6'
when brpx='melvin_12_6' then 'topcraft_12_6'
when brpx='leftcoast_12_6' then 'topcraft_12_6'
when brpx='twopitchers_12_6' then 'topcraft_12_6'
when brpx='belchingbeaver_12_6' then 'topcraft_12_6'
when brpx='alesmith_12_6' then 'topcraft_12_6'
when brpx='trackseven_12_6' then 'topcraft_12_6'
when brpx='barrelhouse_12_6' then 'topcraft_12_6'
when brpx='elsegundo_12_6' then 'topcraft_12_6'
when brpx='stone_12_6' then 'topcraft_12_6'
when brpx='ballastpoint_12_6' then 'topcraft_12_6'
when brpx='maui_12_6' then 'topcraft_12_6'
when brpx='elysian_12_6' then 'topcraft_12_6'
when brpx='greenflash_12_6' then 'topcraft_12_6'
when brpx='sixpoint_12_6' then 'topcraft_12_6'
when brpx='epic_12_6' then 'topcraft_12_6'
when brpx='alpine_12_6' then 'topcraft_12_6'
when brpx='seismic_12_6' then 'topcraft_12_6'
when brpx='mammoth_12_6' then 'topcraft_12_6'
when brpx='fw_11.2_6' then 'topcraft_12_6'
when brpx='hubhopworks_12_6' then 'topcraft_12_6'
when brpx='lostabbey_12_6' then 'topcraft_12_6'
when brpx='einstokolgerd_11.2_6' then 'topcraft_12_6'
when brpx='allagash_12_6' then 'topcraft_12_6'
when brpx='kombrewcha_12_6' then 'topcraft_12_6'
when brpx='victory_12_6' then 'topcraft_12_6'
when brpx='celia_11.2_6' then 'topcraft_12_6'
when brpx='newholland_12_6' then 'topcraft_12_6'
when brpx='kernriver_12_6' then 'topcraft_12_6'
when brpx='flyingdog_12_6' then 'topcraft_12_6'
when brpx='juneshine_12_6' then 'topcraft_12_6'
when brpx='crookedstave_12_6' then 'topcraft_12_6'
when brpx='revision_12_6' then 'topcraft_12_6'
when brpx='crux_12_6' then 'topcraft_12_6'
when brpx='jackieos_12_6' then 'topcraft_12_6'
when brpx='redhook_12_12' then 'natcraft_12_12'
when brpx='founders_12_12' then 'natcraft_12_12'
when brpx='gooseisland_12_12' then 'natcraft_12_12'
when brpx='shocktop_12_12' then 'natcraft_12_12'
when brpx='housebeer_12_12' then 'locraft_12_12'
when brpx='leinenkugels_12_12' then 'natcraft_12_12'
when brpx='marathon_12_12' then 'locraft_12_12'
when brpx='lagunitas_12_12' then 'natcraft_12_12'
when brpx='firestone_12_12' then 'locraft_12_12'
when brpx='lostcoast_12_12' then 'locraft_12_12'
when brpx='kona_12_12' then 'natcraft_12_12'
when brpx='oskarblues_12_12' then 'locraft_12_12'
when brpx='hangar24_12_12' then 'locraft_12_12'
when brpx='newbelgium_12_12' then 'natcraft_12_12'
when brpx='omission_12_12' then 'locraft_12_12'
when brpx='anchor_12_12' then 'locraft_12_12'
when brpx='bluemoon_12_12' then 'natcraft_12_12'
when brpx='hopvalley_12_12' then 'locraft_12_12'
when brpx='widmerbrothers_12_12' then 'locraft_12_12'
when brpx='sierranevada_12_12' then 'natcraft_12_12'
when brpx='samueladams_12_12' then 'natcraft_12_12'
when brpx='saintarcher_12_12' then 'locraft_12_12'
when brpx='steelhead_12_12' then 'locraft_12_12'
when brpx='northcoast_12_12' then 'locraft_12_12'
when brpx='karlstrauss_12_12' then 'locraft_12_12'
when brpx='bearrepublic_12_12' then 'locraft_12_12'
when brpx='ninkasi_12_12' then 'locraft_12_12'
when brpx='coronado_12_12' then 'locraft_12_12'
when brpx='deschutes_12_12' then 'locraft_12_12'
when brpx='stone_12_12' then 'topcraft_12_12'
when brpx='figueroamountain_12_12' then 'topcraft_12_12'
when brpx='alaskan_12_12' then 'topcraft_12_12'
when brpx='21stamendment_12_12' then 'topcraft_12_12'
when brpx='brooklyn_12_12' then 'topcraft_12_12'
when brpx='ballastpoint_12_12' then 'topcraft_12_12'
when brpx='motherearthbrewco_12_12' then 'topcraft_12_12'
when brpx='avery_12_12' then 'topcraft_12_12'
when brpx='10barrel_12_12' then 'topcraft_12_12'
when brpx='sixpoint_12_12' then 'topcraft_12_12'
when brpx='alpine_12_12' then 'topcraft_12_12'
when brpx='bells_12_12' then 'topcraft_12_12'
when brpx='elysian_12_12' then 'topcraft_12_12'
when brpx='pizzaport_16_12' then 'topcraft_16_12'
when brpx='alesmith_12_12' then 'topcraft_12_12'
when brpx='allagash_12_12' then 'topcraft_12_12'
when brpx='shocktop_12_15' then 'natcraft_12_15'
when brpx='gooseisland_12_15' then 'natcraft_12_15'
when brpx='goldenroad_12_15' then 'natcraft_12_15'
when brpx='bluemoon_12_15' then 'natcraft_12_15'
when brpx='founders_12_15' then 'natcraft_12_15'
when brpx='oskarblues_12_15' then 'natcraft_12_15'
when brpx='victory_12_15' then 'natcraft_12_15'
when brpx='kona_12_18' then 'natcraft_12_18'
when brpx='firestone_12_18' then 'natcraft_12_18'
when brpx='sierranevada_12_24' then 'natcraft_12_24'
when brpx='elysian_12_24' then 'natcraft_12_24'
when brpx='kona_12_24' then 'natcraft_12_24'
when brpx='shocktop_12_24' then 'natcraft_12_24'
when brpx='bluemoon_12_24' then 'natcraft_12_24'
when brpx='firestone_12_24' then 'natcraft_12_24'
when brpx='smirnoff_11.2_6' then 'fmb_12_6'
when brpx='mikes_11.2_6' then 'fmb_12_6'
when brpx='kyla_11.2_6' then 'fmb_12_6'
when brpx='caymanjack_11.2_6' then 'fmb_12_6'
when brpx='jugofuertemikes_11.2_6' then 'fmb_12_6'
when brpx='purastill_11.2_6' then 'fmb_12_6'
when brpx='smirnoff_11.2_12' then 'fmb_12_12'
when brpx='mikes_11.2_12' then 'fmb_12_12'
when brpx='whiteclawhardseltzer_12_6' then 'seltz_12_6'
when brpx='trulyhardseltzer_12_6' then 'seltz_12_6'
when brpx='coronarefresca_12_6' then 'seltz_12_6'
when brpx='bon&vivspikedseltzer_12_6' then 'seltz_12_6'
when brpx='redds_12_6' then 'fmb_12_6'
when brpx='notyourfathers_12_6' then 'fmb_12_6'
when brpx='capeline_12_6' then 'fmb_12_6'
when brpx='belchingbeaver_12_6' then 'fmb_12_6'
when brpx='vizzy_12_6' then 'seltz_12_6'
when brpx='henrys_12_6' then 'fmb_12_6'
when brpx='seekout_12_6' then 'fmb_12_6'
when brpx='smirnoff_12_6' then 'fmb_12_6'
when brpx='ficks_12_6' then 'fmb_12_6'
when brpx='arnoldpalmer_12_6' then 'fmb_12_6'
when brpx='hardfrescos_12_6' then 'fmb_12_6'
when brpx='wildbasin_12_6' then 'fmb_12_6'
when brpx='eelriver_12_6' then 'fmb_12_6'
when brpx='omission_12_6' then 'fmb_12_6'
when brpx='lqdcreativeliquids_12_6' then 'fmb_12_6'
when brpx='kyla_12_6' then 'fmb_12_6'
when brpx='press_12_6' then 'fmb_12_6'
when brpx='goldenroad_12_6' then 'fmb_12_6'
when brpx='twistedtea_12_6' then 'fmb_12_6'
when brpx='squiela_12_6' then 'fmb_12_6'
when brpx='kona_12_6' and mc_subsegment_c='fmb' then 'fmb_12_6'
when brpx='bestdamn_12_6' then 'fmb_12_6'
when brpx='oculto_12_6' then 'fmb_12_6'
when brpx='crookandmarker_12_8' then 'fmb_12_8'
when brpx='kyla_12_8' then 'fmb_12_8'
when brpx='malibu_12_8' then 'fmb_12_8'
when brpx='whiteclawhardseltzer_12_12' then 'seltz_12_12'
when brpx='trulyhardseltzer_12_12' then 'seltz_12_12'
when brpx='corona_12_12' then 'seltz_12_12'
when brpx='budlight_12_12' and mc_subsegment_c='fmb' then 'seltz_12_12'
when brpx='smirnoff_12_12' then 'fmb_12_12'
when brpx='bon&vivspikedseltzer_12_12' then 'seltz_12_12'
when brpx='mikes_12_12' then 'fmb_12_12'
when brpx='coronarefresca_12_12' then 'fmb_12_12'
when brpx='goldenroad_12_12' then 'fmb_12_12'
when brpx='vizzy_12_12' then 'seltz_12_12'
when brpx='maha_12_12' then 'fmb_12_12'
when brpx='capeline_12_12' then 'fmb_12_12'
when brpx='redds_12_12' then 'fmb_12_12'
when brpx='arnoldpalmer_12_12' then 'fmb_12_12'
when brpx='ritas_12_12' then 'fmb_12_12'
when brpx='natural_12_12' and mc_subsegment_c='fmb' then 'seltz_12_12'
when brpx='twistedtea_12_12' then 'fmb_12_12'
when brpx='wildbasin_12_12' then 'fmb_12_12'
when brpx='henrys_12_12' then 'fmb_12_12'
when brpx='purastill_12_12' then 'fmb_12_12'
when brpx='omission_12_12' then 'fmb_12_12'
when brpx='ficks_12_12' then 'fmb_12_12'
when brpx='press_12_12' then 'fmb_12_12'
when brpx='seekout_12_12' then 'fmb_12_12'
when brpx='arcticsummer_12_12' then 'fmb_12_12'
when brpx='budlight_12_24' and mc_subsegment_c='fmb' then 'seltz_12_24'
when brpx='trulyhardseltzer_12_24' then 'seltz_12_24'
when brpx='whiteclawhardseltzer_12_24' then 'seltz_12_24'
when brpx='heineken_11.2_6' then 'euro_12_6'
when brpx='stellaartois_11.2_6' then 'euro_12_6'
when brpx='peroni_11.2_6' then 'euro_12_6'
when brpx='guinness_11.2_6' then 'euro_12_6'
when brpx='abbeyofleffe_11.2_6' then 'euro_12_6'
when brpx='einstokolgerd_11.2_6' then 'euro_12_6'
when brpx='hoegaarden_11.2_6' then 'euro_12_6'
when brpx='weihenstephan_11.2_6' then 'euro_12_6'
when brpx='bitburger_11.2_6' then 'euro_12_6'
when brpx='sol_11.2_6' then 'mex_12_6'
when brpx='erdinger_11.2_6' then 'euro_12_6'
when brpx='damm_11.2_6' then 'euro_12_6'
when brpx='cusquena_11.2_6' then 'euro_12_6'
when brpx='hofbrau_11.2_6' then 'euro_12_6'
when brpx='carlsberg_11.2_6' then 'euro_12_6'
when brpx='stiegl_11.2_6' then 'euro_12_6'
when brpx='estrella_11.2_6' then 'mex_12_6'
when brpx='bavik_11.2_6' then 'euro_12_6'
when brpx='paulaner_11.2_6' then 'euro_12_6'
when brpx='maltstar_11.2_6' then 'euro_12_6'
when brpx='czechvar_11.2_6' then 'euro_12_6'
when brpx='powermal_11.2_6' then 'euro_12_6'
when brpx='petrus_11.2_6' then 'euro_12_6'
when brpx='piraat_11.2_6' then 'euro_12_6'
when brpx='chang_11.2_6' then 'euro_12_6'
when brpx='stellaartois_11.2_12' then 'euro_12_12'
when brpx='guinness_11.2_12' then 'euro_12_12'
when brpx='sol_11.2_12' then 'mex_12_12'
when brpx='peroni_11.2_12' then 'euro_12_12'
when brpx='hoegaarden_11.2_12' then 'euro_12_12'
when brpx='sanmiguel_11.2_12' then 'mex_12_12'
when brpx='stellaartois_11.2_18' then 'euro_12_18'
when brpx='stellaartois_11.2_24' then 'euro_12_24'
when brpx='heineken_11.2_24' then 'euro_12_24'
when brpx='guinness_11.2_24' then 'euro_12_24'
when brpx='modelo_12_6' then 'mex_12_6'
when brpx='corona_12_6' then 'mex_12_6'
when brpx='pacifico_12_6' then 'mex_12_6'
when brpx='heineken_12_6' then 'euro_12_6'
when brpx='becks_12_6' then 'euro_12_6'
when brpx='dosequis_12_6' then 'mex_12_6'
when brpx='guinness_12_6' then 'euro_12_6'
when brpx='stpauligirl_12_6' then 'euro_12_6'
when brpx='estrellajalisco_12_6' then 'mex_12_6'
when brpx='newcastle_12_6' then 'euro_12_6'
when brpx='pilsnerurquell_12_6' then 'euro_12_6'
when brpx='amstel_12_6' then 'euro_12_6'
when brpx='weihenstephan_12_6' then 'euro_12_6'
when brpx='victoria_12_6' then 'euro_12_6'
when brpx='cartablanca_12_6' then 'mex_12_6'
when brpx='tecate_12_6' then 'prem_12_6'
when brpx='kirin_12_6' then 'euro_12_6'
when brpx='spaten_12_6' then 'euro_12_6'
when brpx='paulaner_12_6' then 'euro_12_6'
when brpx='harp_12_6' then 'euro_12_6'
when brpx='smithwicks_12_6' then 'euro_12_6'
when brpx='franziskaner_12_6' then 'euro_12_6'
when brpx='bohemia_12_6' then 'euro_12_6'
when brpx='famosa_12_6' then 'euro_12_6'
when brpx='suprema_12_6' then 'euro_12_6'
when brpx='buckler_12_6' then 'euro_12_6'
when brpx='cervezapalma_12_6' then 'mex_12_6'
when brpx='kronenbourg_12_6' then 'euro_12_6'
when brpx='clausthaler_12_6' then 'euro_12_6'
when brpx='imperial_12_6' then 'euro_12_6'
when brpx='kaliber_12_6' then 'euro_12_6'
when brpx='cristal_12_6' then 'euro_12_6'
when brpx='bajabrewing_12_6' then 'mex_12_6'
when brpx='stellaartois_12_6' then 'euro_12_6'
when brpx='bass_12_6' then 'euro_12_6'
when brpx='hofbrau_12_6' then 'euro_12_6'
when brpx='pilsener_12_6' then 'euro_12_6'
when brpx='quilmes_12_6' then 'euro_12_6'
when brpx='cervezaurbana_12_6' then 'mex_12_6'
when brpx='unibroue_12_6' then 'euro_12_6'
when brpx='morland_12_6' then 'euro_12_6'
when brpx='rrey_12_6' then 'euro_12_6'
when brpx='modelo_12_12' then 'mex_12_12'
when brpx='corona_12_12' then 'mex_12_12'
when brpx='pacifico_12_12' then 'mex_12_12'
when brpx='heineken_12_12' then 'euro_12_12'
when brpx='dosequis_12_12' then 'mex_12_12'
when brpx='tecate_12_12' then 'prem_12_12'
when brpx='estrellajalisco_12_12' then 'mex_12_12'
when brpx='victoria_12_12' then 'euro_12_12'
when brpx='newcastle_12_12' then 'euro_12_12'
when brpx='bohemia_12_12' then 'euro_12_12'
when brpx='montejo_12_12' then 'mex_12_12'
when brpx='becks_12_12' then 'euro_12_12'
when brpx='stellaartois_12_12' then 'euro_12_12'
when brpx='cartablanca_12_12' then 'mex_12_12'
when brpx='famosa_12_12' then 'mex_12_12'
when brpx='cervezasdemexico_12_12' then 'mex_12_12'
when brpx='pilsnerurquell_12_12' then 'euro_12_12'
when brpx='guinness_12_12' then 'euro_12_12'
when brpx='stpauligirl_12_12' then 'euro_12_12'
when brpx='mexicali_12_12' then 'mex_12_12'
when brpx='amstel_12_12' then 'euro_12_12'
when brpx='sol_12_12' then 'mex_12_12'
when brpx='pilsener_12_12' then 'euro_12_12'
when brpx='franziskaner_12_12' then 'euro_12_12'
when brpx='bass_12_12' then 'euro_12_12'
when brpx='kirin_12_12' then 'euro_12_12'
when brpx='estrellajalisco_12_15' then 'mex_12_15'
when brpx='modelo_12_18' then 'mex_12_18'
when brpx='corona_12_18' then 'mex_12_18'
when brpx='tecate_12_18' then 'prem_12_18'
when brpx='dosequis_12_18' then 'mex_12_18'
when brpx='heineken_12_18' then 'euro_12_18'
when brpx='montejo_12_18' then 'mex_12_18'
when brpx='tecate_12_20' then 'prem_12_20'
when brpx='modelo_12_24' then 'mex_12_24'
when brpx='corona_12_24' then 'mex_12_24'
when brpx='pacifico_12_24' then 'mex_12_24'
when brpx='heineken_12_24' then 'euro_12_24'
when brpx='dosequis_12_24' then 'mex_12_24'
when brpx='stpauligirl_12_24' then 'euro_12_24'
when brpx='victoria_12_24' then 'euro_12_24'
when brpx='cervezasdemexico_12_24' then 'mex_12_24'
when brpx='tecate_12_30' then 'prem_12_30'
when brpx='montejo_12_30' then 'mex_12_30'
when brpx='tecate_12_36' then 'prem_12_36'
when brpx='miller_12_6' and mc_subsegment_c='near premium' then 'econ_12_6'
when brpx='rollingrock_12_6' then 'econ_12_6'
when brpx='pabst_12_6' then 'econ_12_6'
when brpx='miller_12_12' and mc_subsegment_c='near premium' then 'econ_12_12'
when brpx='pabst_12_12' then 'econ_12_12'
when brpx='rollingrock_12_12' then 'econ_12_12'
when brpx='busch_12_12' then 'econ_12_12'
when brpx='pabst_12_18' then 'econ_12_18'
when brpx='miller_12_24' and mc_subsegment_c='near premium' then 'econ_12_24'
when brpx='rollingrock_12_30' then 'econ_12_30'
when brpx='pabst_12_30' then 'econ_12_30'
when brpx='busch_12_30' then 'econ_12_30'
when brpx='miller_16_6' and mc_subsegment_c='near premium' then 'econ_16_6'
when brpx='pabst_16_6' then 'econ_16_6'
when brpx='budlight_12_6' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_6'
when brpx='budlight_12_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_12'
when brpx='budlight_12_18' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_18'
when brpx='budlight_12_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_20'
when brpx='budlight_12_24' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_24'
when brpx='budlight_12_30' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_30'
when brpx='budlight_12_36' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_36'
when brpx='budlight_16_8' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_8'
when brpx='budlight_16_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_12'
when brpx='budlight_16_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_20'
when brpx='budweiser_12_30' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_30'
when brpx='budweiser_12_6' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_6'
when brpx='budweiser_12_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_12'
when brpx='budweiser_12_18' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_18'
when brpx='budweiser_12_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_20'
when brpx='budweiser_12_30' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_30'
when brpx='budweiser_12_36' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_36'
when brpx='budweiser_16_6' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_6'
when brpx='budweiser_16_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_12'
when brpx='budweiser_16_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_20'
when brpx='coors_12_6' then 'prem_12_6'
when brpx='coors_12_12' then 'prem_12_12'
when brpx='coors_12_18' then 'prem_12_18'
when brpx='coors_12_20' then 'prem_12_20'
when brpx='coors_12_30' then 'prem_12_30'
when brpx='coors_12_36' then 'prem_12_36'
when brpx='coors_16_9' then 'prem_16_9'
when brpx='coors_16_15' then 'prem_16_15'
when brpx='coors_12_6' then 'prem_12_6'
when brpx='coors_12_12' then 'prem_12_12'
when brpx='coors_12_18' then 'prem_12_18'
when brpx='coors_12_20' then 'prem_12_20'
when brpx='coors_12_30' then 'prem_12_30'
when brpx='coors_12_36' then 'prem_12_36'
when brpx='coors_16_9' then 'prem_16_9'
when brpx='miller_12_6' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_6'
when brpx='miller_12_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_12'
when brpx='miller_12_18' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_18'
when brpx='miller_12_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_20'
when brpx='miller_12_30' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_30'
when brpx='miller_12_36' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_36'
when brpx='miller_16_9' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_9'
when brpx='miller_16_15' and mc_subsegment_c in('premium light','premium regular') then 'prem_16_15'
when brpx='miller_12_6' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_6'
when brpx='miller_12_12' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_12'
when brpx='miller_12_18' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_18'
when brpx='miller_12_20' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_20'
when brpx='miller_12_30' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_30'
when brpx='miller_12_36' and mc_subsegment_c in('premium light','premium regular') then 'prem_12_36'
when brpx='odouls_12_6' then 'prem_12_6'
when brpx='odouls_12_12' then 'prem_12_12'
when brpx='odouls_12_24' then 'prem_12_24'
when brpx='michelobultra_12_6' then 'super_12_6'
when brpx='budlight_12_6' and mc_subsegment_c in('super premium') then 'super_12_6'
when brpx='redbridge_12_6' then 'super_12_6'
when brpx='budweiser_12_6' and mc_subsegment_c in('super premium') then 'super_12_6'
when brpx='budweiser_12_8' and mc_subsegment_c in('super premium') then 'super_12_8'
when brpx='michelobultra_12_12' then 'super_12_12'
when brpx='budlight_12_12' then 'super_12_12'
when brpx='michelob_12_12' then 'super_12_12'
when brpx='budweiser_12_12' and mc_subsegment_c in('super premium') then 'super_12_12'
when brpx='landshark_12_12' then 'super_12_12'
when brpx='michelobultra_12_18' then 'super_12_18'
when brpx='michelobultra_12_20' then 'super_12_20'
when brpx='michelobultra_12_24' then 'super_12_24'
 else brpx
end as brpx2"""

# Define staging table query to create temp table in your personal Impala database

In [19]:
#define SQL query
pricing_coefficients_staging_query = """
create table """ + your_db + """.pricing_coefficients_staging as 

select 
tdoutletcd
, weekendingdate
, sum(units) as units
, sum(dollar) as rev
, sum(cseq_vol) as vol
, sum(case when mc_subsegment_c in ('"""+subsegment_names+"""' ) Then dollar Else 0 End ) as dep_rev
, sum(case when mc_subsegment_c in ('"""+subsegment_names+"""' ) Then units Else 0 End ) as dep_units
, sum(case when mc_subsegment_c in ('"""+subsegment_names+"""' ) Then cseq_vol Else 0 End ) as dep_vol
, sum(case when mc_subsegment_c not in ('"""+subsegment_names+"""' ) Then dollar Else 0 End ) as ao_rev
, sum(case when mc_subsegment_c not in ('"""+subsegment_names+"""' ) Then units Else 0 End ) as ao_units
, sum(case when mc_subsegment_c not in ('"""+subsegment_names+"""' ) Then cseq_vol Else 0 End ) as ao_vol
, count(case when mc_subsegment_c in ('"""+subsegment_names+"""' ) Then upc Else NULL END) as dep_asrt

, REPLACE(brpx2,'&','_') AS brpx

from 
(

select *, """ + brpx_renaming + """

from ( 
select * from (
select *
, (units*pack_size*ounces)/228 as cseq_vol
, concat(brfam,'_',cast(ounces as string),'_',cast(pack_size as string)) as brpx

from (

select 
round(cast(ounces as float)) as 'ounces'
, cast(units as int) as 'units'
, cast(pack_size as int) as 'pack_size'
, lower(brfam) as brfam
, state_abbreviation
--, channel
, store_name
, mc_subsegment_c
, mc_brewer_c
, mc_brand_family_c
, mc_brand_c
, mc_package_size_c
, mc_ounce_c 
, mc_region_of_origin_c
, mc_container_c
, mc_container_material_c
, long_product_description
, tdoutletcd
, weekendingdate
, upc
, dollar
, holiday
, planningunitnme
from 
(
SELECT REPLACE(mc_package_size_c,'pk','') AS pack_size,
REPLACE(mc_ounce_c,'oz','') AS ounces,
REPLACE(replace(mc_brand_family_c,"'",''),' ','') AS brfam
, *
FROM 
(
select 
b.state_abbreviation, 
b.channel, 
b.store_name, 
c.mc_subsegment_c, 
c.mc_brewer_c, 
c.mc_brand_family_c, 
c.mc_brand_c, 
c.mc_package_size_c, 
c.mc_ounce_c, 
c.mc_region_of_origin_c, 
c.mc_container_c, 
c.mc_container_material_c, 
c.long_product_description, 
a.tdoutletcd, 
a.weekendingdate, 
a.upc, 
a.dollar, 
a.units, 
d.holiday, 
e.planningunitnme
from ( 
    select * 
    from ( 
        select distinct 
        tdoutletcd, 
        upc, 
        cast(unix_timestamp(period_description_short, 'yyyy-MM-dd') as timestamp) as weekendingdate, 
        units, 
        dollar
        from core.acn_stc_outlet 
        ) as step2 
    where weekendingdate >= adddate(cast(unix_timestamp('"""+ weekendingdate_start + """', 'MM/dd/yy') as timestamp),(-7*(""" + str(num_weeks) + """ -1))) 
    and weekendingdate <= cast(unix_timestamp('"""+ weekendingdate_start + """', 'MM/dd/yy') as timestamp) 
    ) as a 
inner join 
    (select distinct 
    state_abbreviation, 
    tdlinx_store_code, 
    channel, 
    store_name 
    from core.acn_outlet_dim 
    where  channel like '"""+ channel +"""' and dma_name in ('"""+dma_names+"""' )) as b 
on a.tdoutletcd=b.tdlinx_store_code 
left join 
core.acn_product_dim as c 
on a.upc=c.upc 
left join (select cast(unix_timestamp(week_end, 'M/d/yyyy') as timestamp) as hweek, ind as holiday from core.wk_ending_sat_beer_ind) as d 
on a.weekendingdate=d.hweek 
left join 
(select distinct r.tdoutletcd, d.planningunitnme 
from core.retail_outlet r 
inner join core.str s 
on r.retailoutletid=s.retailoutletid 
inner join core.drmdim d 
on s.distributorretailmarketid = d.distributorretailmarketid 
) 
as e 
on a.tdoutletcd=e.tdoutletcd 
where units > 0 
) g

) aa
) bb

where
pack_size IS NOT NULL
and ounces IS NOT NULL

) cc
where cseq_vol > 0 

UNION
-- create kroger table

select * from (
select 
*
, (units*pack_size*ounces)/228 as cseq_vol
, concat(brfam,'_',cast(ounces as string),'_',cast(pack_size as string)) as brpx

from (
select 
round(cast(ounces as float)) as 'ounces'
, cast(scanned_movement as int) as 'units'
, cast(pack_size as int) as 'pack_size'
, lower(brfam) as brfam
, store_state as 'state_abbreviation'
, store_banner as 'store_name'
, mc_subsegment_c
, mc_brewer_c
, mc_brand_family_c
, mc_brand_c
, mc_package_size_c
, mc_ounce_c 
, mc_region_of_origin_c
, mc_container_c
, mc_container_material_c
, long_product_description
, cast(tdlinx as string) as 'tdoutletcd'
, calendarfiscalwe as 'weekendingdate'
, upc
, scanned_retail_dollars as 'dollar'
, holiday
, planningunitnme
from
(
select 
REPLACE(mc_package_size_c,'pk','') AS pack_size,
REPLACE(mc_ounce_c,'oz','') AS ounces,
REPLACE(mc_brand_family_c,' ','') AS brfam
, *
from 

(select
z.*,
y.holiday,
x.planningunitnme
from

(
select
a.rpt_short_desc,
a.re_sto_num,
a.upc,
a.scanned_retail_dollars,
a.scanned_movement,
a.calendarfiscalwe,
b.long_product_description,
b.mc_brand_family_c,
b.mc_brand_c,
b.mc_region_of_origin_c,
b.mc_brewer_c,
b.mc_subsegment_c,
b.mc_package_size_c,
b.mc_ounce_c,
b.mc_container_c,
b.mc_container_material_c,
c.store_state,
e.tdlinx,
e.store_banner
from 
(
select a.*,cast(unix_timestamp(d.calendarfiscalwe, "M/d/yy") as timestamp) as calendarfiscalwe from
retailer_data_collection_kroger.kroger_ad_sales as a
left join retailer_data_collection_kroger.kroger_date_dim as d
on a.week_name=d.krogerlong
having 
calendarfiscalwe >= adddate(cast(unix_timestamp('""" + weekendingdate_start + """', 'MM/dd/yy') as timestamp),(-7*(""" + str(num_weeks) + """-1)))
and calendarfiscalwe <= cast(unix_timestamp('""" + weekendingdate_start + """', 'MM/dd/yy') as timestamp)
) as a
left join retailer_data_collection_kroger.kroger_outlet_dim as e
on a.rpt_short_desc=e.commondivision
and a.re_sto_num=e.re_sto_num
left join core.acn_product_dim as b
on a.upc=b.upc
left join (select distinct re_sto_num, store_state, commondivision from retailer_data_collection_kroger.kroger_outlet_dim) as c
on a.re_sto_num=c.re_sto_num
and a.rpt_short_desc=c.commondivision

) as z
left join (select cast(unix_timestamp(week_end, 'M/d/yyyy') as timestamp) as hweek, ind as holiday from core.wk_ending_sat_beer_ind) as y
on z.calendarfiscalwe=y.hweek
left join
(select distinct cast(r.tdoutletcd as int) as tdoutletcd, d.planningunitnme
from core.retail_outlet r
inner join core.str s
on r.retailoutletid=s.retailoutletid
inner join core.drmdim d
on s.distributorretailmarketid = d.distributorretailmarketid
) as x
on z.tdlinx=x.tdoutletcd ) ii

where planningunitnme in ('"""+planning_units+"""')) zz
) vv

where
pack_size IS NOT NULL
and ounces IS NOT NULL

) uu
where cseq_vol > 0

) tt

) ss

group by tdoutletcd, weekendingdate, brpx

;
"""

# Define delete staging table query

In [20]:
delete_pricing_coefficients_staging_query = """
drop table if exists """ + your_db + """.pricing_coefficients_staging;
"""

# Delete staging table if exists

In [21]:
cur = cnxn.cursor()
cur.execute(delete_pricing_coefficients_staging_query)
cnxn.commit()

# Run staging table query

In [22]:
cur = cnxn.cursor()
cur.execute(pricing_coefficients_staging_query)
cnxn.commit()

# First create dataframe with totals for each store by week

In [23]:
totals_df = impala_select(cnxn, """select tdoutletcd
, weekendingdate
, sum(units) as total_units
, sum(rev) as total_rev
, sum(vol) as total_vol
from """+ your_db+""".pricing_coefficients_staging
group by 
tdoutletcd
, weekendingdate""")

      tdoutletcd weekendingdate  total_units     total_rev    total_vol
0        0276183     2020-05-30         2003  23353.679855  1129.228070
1        0120547     2018-07-07         3311  42250.700017  2298.798246
2        0919929     2018-10-20         1111  11478.429949   485.504386
3        0120890     2019-01-05         1679  17882.699920   793.956140
4        7451537     2020-02-29         1298  10818.149945   283.004386
...          ...            ...          ...           ...          ...
70282    0120644     2018-09-08         2873  35917.839816  2015.017544
70283    0580908     2019-07-06         3469  46686.909751  2584.767544
70284    0118438     2020-02-15          989  10704.939865   477.859649
70285    0561429     2020-05-02          740  12386.999911   636.899123
70286    0119656     2019-01-05         1955  21334.959924  1250.991228

[70287 rows x 5 columns]


# Then get list (or define list) of brpx variables for table pivoting, allowing us to get totals by store by week for all chosen brpx variables in one row with lots of columns, instead of the other way around

In [24]:
#Grab brpx list automatically
#brpx = impala_select(cnxn, """select distinct brpx from (select tdoutletcd
#, weekendingdate
#, units
#, rev
#, vol
#, REPLACE(brpx,'-','_') AS brpx from """+ your_db+""".pricing_coefficients_staging) a """)
#brpx = list(brpx['brpx'])

#Define list manually
brpx = ['econ_12_12','econ_12_15','econ_12_18','econ_12_24','econ_12_30','econ_16_6',
        'cider_12_6','cider_12_12','cider_12_24','locraft_12_6','natcraft_12_6','topcraft_12_6',
        'natcraft_12_12','locraft_12_12','topcraft_12_12','topcraft_16_12','natcraft_12_15','natcraft_12_18',
        'natcraft_12_24','fmb_12_6','fmb_12_12','seltz_12_6','fmb_12_8','seltz_12_12','seltz_12_24',
        'euro_12_6','mex_12_6','euro_12_12','mex_12_12','euro_12_18','euro_12_24','prem_12_6','prem_12_12',
        'mex_12_15','mex_12_18','prem_12_18','prem_12_20','mex_12_24','prem_12_30','mex_12_30','prem_12_36',
        'econ_12_6','prem_12_24','prem_16_8','prem_16_12','prem_16_20','prem_16_6','prem_16_9','prem_16_15',
        'super_12_6','super_12_8','super_12_12','super_12_18','super_12_20','super_12_24']

# Convert list to CASE WHEN statements for SQL pull

In [25]:
units_vars = list()
for i in brpx:
    units_vars.append("SUM(CASE WHEN brpx = '"+ i +"' THEN units ELSE 0 END) AS " + i + "_units")

sum_units_vars_string = ", ".join(units_vars)

rev_vars = list()
for i in brpx:
    rev_vars.append("SUM(CASE WHEN brpx = '"+ i +"' THEN rev ELSE 0 END) AS " + i + "_rev")

sum_rev_vars_string = ", ".join(rev_vars)

vol_vars = list()
for i in brpx:
    vol_vars.append("SUM(CASE WHEN brpx = '"+ i +"' THEN vol ELSE 0 END) AS " + i + "_vol")

sum_vol_vars_string = ", ".join(vol_vars)

# Define pseudo-pivot sql pull

In [26]:
brpx_list = "', '".join(brpx)

pivot_query = """ SELECT tdoutletcd
, weekendingdate
, sum(units) as 'total_focus_units'
, sum(rev) as 'total_focus_rev'
, sum(vol) as 'total_focus_vol'
, sum(dep_rev) as 'dep_rev'
, sum(dep_asrt) as 'dep_asrt'
, sum(dep_units) as 'dep_units'
, sum(dep_vol) as 'dep_vol'
, sum(ao_vol) as 'ao_vol'
, sum(ao_rev) as 'ao_rev'
, sum(ao_units) as 'ao_units'
, sum(ao_rev)/sum(ao_units) as 'ao_price'
,"""+ sum_units_vars_string +""",""" + sum_rev_vars_string + """,""" + sum_vol_vars_string +""" FROM
(select tdoutletcd, weekendingdate, units, rev, vol,dep_rev,dep_asrt,dep_units,dep_vol,ao_vol,ao_rev,ao_units
, REPLACE(brpx,'-','_') AS brpx from """+ your_db + """.pricing_coefficients_staging)s  
where brpx in ('""" + brpx_list + """')
GROUP BY tdoutletcd, weekendingdate 
"""

In [27]:
pivot_query

" SELECT tdoutletcd\n, weekendingdate\n, sum(units) as 'total_focus_units'\n, sum(rev) as 'total_focus_rev'\n, sum(vol) as 'total_focus_vol'\n, sum(dep_rev) as 'dep_rev'\n, sum(dep_asrt) as 'dep_asrt'\n, sum(dep_units) as 'dep_units'\n, sum(dep_vol) as 'dep_vol'\n, sum(ao_vol) as 'ao_vol'\n, sum(ao_rev) as 'ao_rev'\n, sum(ao_units) as 'ao_units'\n, sum(ao_rev)/sum(ao_units) as 'ao_price'\n,SUM(CASE WHEN brpx = 'econ_12_12' THEN units ELSE 0 END) AS econ_12_12_units, SUM(CASE WHEN brpx = 'econ_12_15' THEN units ELSE 0 END) AS econ_12_15_units, SUM(CASE WHEN brpx = 'econ_12_18' THEN units ELSE 0 END) AS econ_12_18_units, SUM(CASE WHEN brpx = 'econ_12_24' THEN units ELSE 0 END) AS econ_12_24_units, SUM(CASE WHEN brpx = 'econ_12_30' THEN units ELSE 0 END) AS econ_12_30_units, SUM(CASE WHEN brpx = 'econ_16_6' THEN units ELSE 0 END) AS econ_16_6_units, SUM(CASE WHEN brpx = 'cider_12_6' THEN units ELSE 0 END) AS cider_12_6_units, SUM(CASE WHEN brpx = 'cider_12_12' THEN units ELSE 0 END) AS ci

# Create pivoted dataframe

In [28]:
brpx_pivot_df = impala_select(cnxn, pivot_query)

      tdoutletcd weekendingdate  total_focus_units  total_focus_rev  \
0        0276183     2020-05-30               1047     18476.759855   
1        0120547     2018-07-07               2266     36346.250028   
2        0919929     2018-10-20                615      8565.869968   
3        0120890     2019-01-05                970     13781.249931   
4        7451537     2020-02-29                447      5214.879961   
...          ...            ...                ...              ...   
70273    0120644     2018-09-08               1941     31565.119807   
70274    0580908     2019-07-06               2373     40356.509745   
70275    0118438     2020-02-15                549      8276.099884   
70276    0561429     2020-05-02                653     11544.469927   
70277    0119656     2019-01-05               1246     17974.679918   

       total_focus_vol      dep_rev  dep_asrt  dep_units     dep_vol  \
0           925.701754  2856.489998        27        203  194.280702   
1  

In [29]:
brpx_pivot_df.head()

,tdoutletcd,weekendingdate,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,dep_units,dep_vol,ao_vol,...,prem_16_20_vol,prem_16_6_vol,prem_16_9_vol,prem_16_15_vol,super_12_6_vol,super_12_8_vol,super_12_12_vol,super_12_18_vol,super_12_20_vol,super_12_24_vol
0,0276183,2020-05-30,1047,18476.759855,925.701754,2856.489998,27,203,194.280702,731.421053,...,0.0,0.0,0.631579,0.000000,0.000000,0.0,18.315789,4.736842,0.0,56.842105
1,0120547,2018-07-07,2266,36346.250028,2066.894737,6185.749949,47,425,501.578947,1565.315789,...,0.0,0.0,1.894737,6.315789,9.157895,0.0,42.315789,4.736842,0.0,30.315789
2,0919929,2018-10-20,615,8565.869968,393.736842,1830.750011,33,120,130.315789,263.421053,...,0.0,0.0,1.894737,4.210526,4.105263,0.0,15.157895,0.947368,0.0,6.315789
3,0120890,2019-01-05,970,13781.249931,667.631579,3842.489994,37,251,271.368421,396.263158,...,0.0,0.0,2.526316,6.315789,1.894737,0.0,6.947368,0.000000,0.0,3.789474
4,7451537,2020-02-29,447,5214.879961,171.263158,108.769998,7,13,6.315789,164.947368,...,0.0,0.0,0.000000,0.000000,1.263158,0.0,1.894737,0.000000,0.0,0.000000


# Combine totals_df and pivoted brpx totals df

In [30]:
modeling_df = pd.merge(totals_df, brpx_pivot_df, on=['tdoutletcd','weekendingdate'])

In [31]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,prem_16_20_vol,prem_16_6_vol,prem_16_9_vol,prem_16_15_vol,super_12_6_vol,super_12_8_vol,super_12_12_vol,super_12_18_vol,super_12_20_vol,super_12_24_vol
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,0.0,0.0,0.631579,0.000000,0.000000,0.0,18.315789,4.736842,0.0,56.842105
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,0.0,0.0,1.894737,6.315789,9.157895,0.0,42.315789,4.736842,0.0,30.315789
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,0.0,0.0,1.894737,4.210526,4.105263,0.0,15.157895,0.947368,0.0,6.315789
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,0.0,0.0,2.526316,6.315789,1.894737,0.0,6.947368,0.000000,0.0,3.789474
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,0.0,0.0,0.000000,0.000000,1.263158,0.0,1.894737,0.000000,0.0,0.000000


In [32]:
modeling_df.shape

(70278, 181)

# Create price per unit variable for each brpx

In [33]:
modeling_df.head(50)

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,prem_16_20_vol,prem_16_6_vol,prem_16_9_vol,prem_16_15_vol,super_12_6_vol,super_12_8_vol,super_12_12_vol,super_12_18_vol,super_12_20_vol,super_12_24_vol
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,0.000000,0.0,0.631579,0.000000,0.000000,0.0,18.315789,4.736842,0.0,56.842105
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,0.000000,0.0,1.894737,6.315789,9.157895,0.0,42.315789,4.736842,0.0,30.315789
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,0.000000,0.0,1.894737,4.210526,4.105263,0.0,15.157895,0.947368,0.0,6.315789
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,0.000000,0.0,2.526316,6.315789,1.894737,0.0,6.947368,0.000000,0.0,3.789474
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,0.000000,0.0,0.000000,0.000000,1.263158,0.0,1.894737,0.000000,0.0,0.000000
5,0953459,2019-06-15,1890,17592.560041,906.684211,673,12402.770091,660.824561,1300.019990,23,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.263158,3.789474,0.0,7.578947
6,0561425,2020-05-30,1652,25145.369924,1318.578947,1302,22867.069931,1232.561404,4782.499983,30,...,1.403509,0.0,0.000000,4.210526,0.000000,0.0,10.105263,4.736842,0.0,70.736842
7,0560794,2019-08-03,897,13617.879942,675.109649,698,11734.469938,595.526316,1770.849976,24,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.789474,0.000000,0.0,10.105263
8,0118009,2018-11-10,1035,10765.439907,479.289474,588,8128.639925,387.421053,1971.489975,35,...,0.000000,0.0,3.157895,2.105263,2.842105,0.0,7.578947,0.947368,0.0,2.526316
9,0120580,2019-06-08,2079,31932.800015,1836.184211,1570,29139.990032,1727.754386,7668.110069,47,...,11.228070,0.0,1.894737,3.157895,0.631579,0.0,18.315789,2.842105,0.0,10.105263


In [34]:
modeling_df['dep_asrt'] = modeling_df['dep_asrt'].replace(0, 1)    

In [35]:
for i in brpx:
    pricing_var = i + '_price'
    units_var = i + '_units'
    rev_var = i + '_rev'
    modeling_df[pricing_var] = modeling_df[rev_var]/modeling_df[units_var]

In [36]:
modeling_df = modeling_df.fillna(0)
for i in brpx:
    pricing_var = i + '_price'
    max_price = i + '_max'
    modeling_df[max_price]=max(modeling_df[pricing_var])
    modeling_df[pricing_var] = np.where(modeling_df[pricing_var] == 0, modeling_df[max_price], modeling_df[pricing_var])

In [37]:
modeling_df = modeling_df[modeling_df.columns.drop(list(modeling_df.filter(regex='_max')))]

In [38]:
modeling_df.head(500)

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,prem_16_20_price,prem_16_6_price,prem_16_9_price,prem_16_15_price,super_12_6_price,super_12_8_price,super_12_12_price,super_12_18_price,super_12_20_price,super_12_24_price
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,23.990000,12.99,10.990000,20.990000,13.490000,14.99,13.851724,16.989999,17.99,20.144666
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,23.990000,12.99,11.990000,14.990000,8.369310,14.99,11.333284,17.989999,17.99,19.990000
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,23.990000,12.99,12.990000,17.990000,7.534615,14.99,13.281667,20.990000,17.99,23.990000
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,23.990000,12.99,12.990000,17.990000,8.990000,14.99,13.080909,22.990000,17.99,23.990000
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,23.990000,12.99,15.990000,20.990000,7.192500,14.99,13.656666,22.990000,17.99,36.990002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0565369,2018-09-01,755,10800.169884,601.469298,579,9543.389891,548.596491,2719.779978,28,...,17.990000,12.99,15.990000,14.990000,13.490000,14.99,11.490000,22.990000,17.99,17.990000
496,7527606,2019-08-03,1761,18674.829927,909.135965,1006,14899.359951,774.912281,3207.149992,38,...,21.990000,12.99,12.989999,17.990000,8.156666,14.99,12.434444,16.990000,17.99,19.990000
497,0119807,2018-10-20,1602,17266.449997,970.372807,905,14271.600007,847.824561,5333.640007,47,...,19.989999,12.99,9.990000,14.989999,8.132857,14.99,12.407084,17.990000,17.99,19.990000
498,2229406,2019-07-13,1353,15607.269863,857.210526,870,12938.729889,754.877193,4685.219940,48,...,21.989999,12.99,12.989999,16.562857,8.156667,14.99,13.090816,16.617500,17.99,19.990000


In [39]:
min(modeling_df['prem_12_30_price'])

14.989999740842789

# Create Month and Year variables

In [40]:
modeling_df['year'], modeling_df['month'] = modeling_df['weekendingdate'].dt.year, modeling_df['weekendingdate'].dt.month

In [41]:
modeling_df.head(10)

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,prem_16_9_price,prem_16_15_price,super_12_6_price,super_12_8_price,super_12_12_price,super_12_18_price,super_12_20_price,super_12_24_price,year,month
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,10.990000,20.99,13.490000,14.99,13.851724,16.989999,17.99,20.144666,2020,5
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,11.990000,14.99,8.369310,14.99,11.333284,17.989999,17.99,19.990000,2018,7
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,12.990000,17.99,7.534615,14.99,13.281667,20.990000,17.99,23.990000,2018,10
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,12.990000,17.99,8.990000,14.99,13.080909,22.990000,17.99,23.990000,2019,1
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,15.990000,20.99,7.192500,14.99,13.656666,22.990000,17.99,36.990002,2020,2
5,0953459,2019-06-15,1890,17592.560041,906.684211,673,12402.770091,660.824561,1300.019990,23,...,15.990000,20.99,13.490000,14.99,11.990000,13.970000,17.99,18.975000,2019,6
6,0561425,2020-05-30,1652,25145.369924,1318.578947,1302,22867.069931,1232.561404,4782.499983,30,...,15.990000,16.99,13.490000,14.99,13.177500,15.989999,17.99,19.894286,2020,5
7,0560794,2019-08-03,897,13617.879942,675.109649,698,11734.469938,595.526316,1770.849976,24,...,15.990000,20.99,13.490000,14.99,11.990000,22.990000,17.99,21.492501,2019,8
8,0118009,2018-11-10,1035,10765.439907,479.289474,588,8128.639925,387.421053,1971.489975,35,...,12.989999,17.99,8.878889,14.99,14.240000,20.990000,17.99,23.990000,2018,11
9,0120580,2019-06-08,2079,31932.800015,1836.184211,1570,29139.990032,1727.754386,7668.110069,47,...,12.990000,17.99,7.990000,14.99,12.024483,14.990000,17.99,19.990000,2019,6


##### Convert numerical representation of month and year to categorical

In [42]:
modeling_df['month'] = [calendar.month_abbr[i] for i in modeling_df['month']]

In [43]:
modeling_df['year'] = ['year_'+str(i) for i in modeling_df['year']]

In [44]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,prem_16_9_price,prem_16_15_price,super_12_6_price,super_12_8_price,super_12_12_price,super_12_18_price,super_12_20_price,super_12_24_price,year,month
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,10.99,20.99,13.490000,14.99,13.851724,16.989999,17.99,20.144666,year_2020,May
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,11.99,14.99,8.369310,14.99,11.333284,17.989999,17.99,19.990000,year_2018,Jul
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,12.99,17.99,7.534615,14.99,13.281667,20.990000,17.99,23.990000,year_2018,Oct
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,12.99,17.99,8.990000,14.99,13.080909,22.990000,17.99,23.990000,year_2019,Jan
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,15.99,20.99,7.192500,14.99,13.656666,22.990000,17.99,36.990002,year_2020,Feb


##### One-hot encode month and year variables

In [45]:
modeling_df = pd.get_dummies(modeling_df, columns=['year','month'], prefix ='',prefix_sep='')

In [46]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,0,0,0,0,0,0,1,0,0,0
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,0,0,0,1,0,0,0,0,0,0
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,0,0,0,0,0,0,0,0,1,0
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,0,0,1,0,0,0,0,0,0,0
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,0,1,0,0,0,0,0,0,0,0


# Log variables

##### List out the variables in df to decide on which ones to log

In [47]:
list(modeling_df)

['tdoutletcd',
 'weekendingdate',
 'total_units',
 'total_rev',
 'total_vol',
 'total_focus_units',
 'total_focus_rev',
 'total_focus_vol',
 'dep_rev',
 'dep_asrt',
 'dep_units',
 'dep_vol',
 'ao_vol',
 'ao_rev',
 'ao_units',
 'ao_price',
 'econ_12_12_units',
 'econ_12_15_units',
 'econ_12_18_units',
 'econ_12_24_units',
 'econ_12_30_units',
 'econ_16_6_units',
 'cider_12_6_units',
 'cider_12_12_units',
 'cider_12_24_units',
 'locraft_12_6_units',
 'natcraft_12_6_units',
 'topcraft_12_6_units',
 'natcraft_12_12_units',
 'locraft_12_12_units',
 'topcraft_12_12_units',
 'topcraft_16_12_units',
 'natcraft_12_15_units',
 'natcraft_12_18_units',
 'natcraft_12_24_units',
 'fmb_12_6_units',
 'fmb_12_12_units',
 'seltz_12_6_units',
 'fmb_12_8_units',
 'seltz_12_12_units',
 'seltz_12_24_units',
 'euro_12_6_units',
 'mex_12_6_units',
 'euro_12_12_units',
 'mex_12_12_units',
 'euro_12_18_units',
 'euro_12_24_units',
 'prem_12_6_units',
 'prem_12_12_units',
 'mex_12_15_units',
 'mex_12_18_units',


##### Make a list of all the vars you want to log

In [48]:
vars_to_log = [ 'dep_asrt',
 'dep_units',
 'dep_vol',
 'ao_vol',
 'ao_rev',
 'ao_units',
 'ao_price',
 'econ_12_12_price',
 'econ_12_15_price',
 'econ_12_18_price',
 'econ_12_24_price',
 'econ_12_30_price',
 'econ_16_6_price',
 'cider_12_6_price',
 'cider_12_12_price',
 'cider_12_24_price',
 'locraft_12_6_price',
 'natcraft_12_6_price',
 'topcraft_12_6_price',
 'natcraft_12_12_price',
 'locraft_12_12_price',
 'topcraft_12_12_price',
 'topcraft_16_12_price',
 'natcraft_12_15_price',
 'natcraft_12_18_price',
 'natcraft_12_24_price',
 'fmb_12_6_price',
 'fmb_12_12_price',
 'seltz_12_6_price',
 'fmb_12_8_price',
 'seltz_12_12_price',
 'seltz_12_24_price',
 'euro_12_6_price',
 'mex_12_6_price',
 'euro_12_12_price',
 'mex_12_12_price',
 'euro_12_18_price',
 'euro_12_24_price',
 'prem_12_6_price',
 'prem_12_12_price',
 'mex_12_15_price',
 'mex_12_18_price',
 'prem_12_18_price',
 'prem_12_20_price',
 'mex_12_24_price',
 'prem_12_30_price',
 'mex_12_30_price',
 'prem_12_36_price',
 'econ_12_6_price',
 'prem_12_24_price',
 'prem_16_8_price',
 'prem_16_12_price',
 'prem_16_20_price',
 'prem_16_6_price',
 'prem_16_9_price',
 'prem_16_15_price',
 'super_12_6_price',
 'super_12_8_price',
 'super_12_12_price',
 'super_12_18_price',
 'super_12_20_price',
 'super_12_24_price']

##### Loop through the list and  create logged variables

In [49]:
for i in vars_to_log:
    logged_var = 'log_'+i
    modeling_df[logged_var] = np.log(modeling_df[i])

/jupyterhub_env/lib/python3.5/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [50]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,log_prem_16_20_price,log_prem_16_6_price,log_prem_16_9_price,log_prem_16_15_price,log_super_12_6_price,log_super_12_8_price,log_super_12_12_price,log_super_12_18_price,log_super_12_20_price,log_super_12_24_price
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,3.177637,2.56418,2.396986,3.044046,2.601949,2.707383,2.628410,2.832625,2.889816,3.002940
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,3.177637,2.56418,2.484073,2.707383,2.124571,2.707383,2.427744,2.889816,2.889816,2.995232
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,3.177637,2.56418,2.564180,2.889816,2.019508,2.707383,2.586385,3.044046,2.889816,3.177637
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,3.177637,2.56418,2.564180,2.889816,2.196113,2.707383,2.571154,3.135059,2.889816,3.177637
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,3.177637,2.56418,2.771964,3.044046,1.973039,2.707383,2.614228,3.135059,2.889816,3.610648


In [51]:
list(modeling_df)

['tdoutletcd',
 'weekendingdate',
 'total_units',
 'total_rev',
 'total_vol',
 'total_focus_units',
 'total_focus_rev',
 'total_focus_vol',
 'dep_rev',
 'dep_asrt',
 'dep_units',
 'dep_vol',
 'ao_vol',
 'ao_rev',
 'ao_units',
 'ao_price',
 'econ_12_12_units',
 'econ_12_15_units',
 'econ_12_18_units',
 'econ_12_24_units',
 'econ_12_30_units',
 'econ_16_6_units',
 'cider_12_6_units',
 'cider_12_12_units',
 'cider_12_24_units',
 'locraft_12_6_units',
 'natcraft_12_6_units',
 'topcraft_12_6_units',
 'natcraft_12_12_units',
 'locraft_12_12_units',
 'topcraft_12_12_units',
 'topcraft_16_12_units',
 'natcraft_12_15_units',
 'natcraft_12_18_units',
 'natcraft_12_24_units',
 'fmb_12_6_units',
 'fmb_12_12_units',
 'seltz_12_6_units',
 'fmb_12_8_units',
 'seltz_12_12_units',
 'seltz_12_24_units',
 'euro_12_6_units',
 'mex_12_6_units',
 'euro_12_12_units',
 'mex_12_12_units',
 'euro_12_18_units',
 'euro_12_24_units',
 'prem_12_6_units',
 'prem_12_12_units',
 'mex_12_15_units',
 'mex_12_18_units',


# Model

In [52]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,log_prem_16_20_price,log_prem_16_6_price,log_prem_16_9_price,log_prem_16_15_price,log_super_12_6_price,log_super_12_8_price,log_super_12_12_price,log_super_12_18_price,log_super_12_20_price,log_super_12_24_price
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,3.177637,2.56418,2.396986,3.044046,2.601949,2.707383,2.628410,2.832625,2.889816,3.002940
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,3.177637,2.56418,2.484073,2.707383,2.124571,2.707383,2.427744,2.889816,2.889816,2.995232
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,3.177637,2.56418,2.564180,2.889816,2.019508,2.707383,2.586385,3.044046,2.889816,3.177637
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,3.177637,2.56418,2.564180,2.889816,2.196113,2.707383,2.571154,3.135059,2.889816,3.177637
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,3.177637,2.56418,2.771964,3.044046,1.973039,2.707383,2.614228,3.135059,2.889816,3.610648


In [53]:
modeling_df = modeling_df.replace([np.inf, -np.inf], np.nan)
modeling_df = modeling_df.fillna(0)

In [54]:
np.isfinite(modeling_df.all())

tdoutletcd               True
weekendingdate           True
total_units              True
total_rev                True
total_vol                True
                         ... 
log_super_12_8_price     True
log_super_12_12_price    True
log_super_12_18_price    True
log_super_12_20_price    True
log_super_12_24_price    True
Length: 313, dtype: bool

In [55]:
modeling_df.head()

,tdoutletcd,weekendingdate,total_units,total_rev,total_vol,total_focus_units,total_focus_rev,total_focus_vol,dep_rev,dep_asrt,...,log_prem_16_20_price,log_prem_16_6_price,log_prem_16_9_price,log_prem_16_15_price,log_super_12_6_price,log_super_12_8_price,log_super_12_12_price,log_super_12_18_price,log_super_12_20_price,log_super_12_24_price
0,0276183,2020-05-30,2003,23353.679855,1129.228070,1047,18476.759855,925.701754,2856.489998,27,...,3.177637,2.56418,2.396986,3.044046,2.601949,2.707383,2.628410,2.832625,2.889816,3.002940
1,0120547,2018-07-07,3311,42250.700017,2298.798246,2266,36346.250028,2066.894737,6185.749949,47,...,3.177637,2.56418,2.484073,2.707383,2.124571,2.707383,2.427744,2.889816,2.889816,2.995232
2,0919929,2018-10-20,1111,11478.429949,485.504386,615,8565.869968,393.736842,1830.750011,33,...,3.177637,2.56418,2.564180,2.889816,2.019508,2.707383,2.586385,3.044046,2.889816,3.177637
3,0120890,2019-01-05,1679,17882.699920,793.956140,970,13781.249931,667.631579,3842.489994,37,...,3.177637,2.56418,2.564180,2.889816,2.196113,2.707383,2.571154,3.135059,2.889816,3.177637
4,7451537,2020-02-29,1298,10818.149945,283.004386,447,5214.879961,171.263158,108.769998,7,...,3.177637,2.56418,2.771964,3.044046,1.973039,2.707383,2.614228,3.135059,2.889816,3.610648


In [56]:
list(modeling_df)

['tdoutletcd',
 'weekendingdate',
 'total_units',
 'total_rev',
 'total_vol',
 'total_focus_units',
 'total_focus_rev',
 'total_focus_vol',
 'dep_rev',
 'dep_asrt',
 'dep_units',
 'dep_vol',
 'ao_vol',
 'ao_rev',
 'ao_units',
 'ao_price',
 'econ_12_12_units',
 'econ_12_15_units',
 'econ_12_18_units',
 'econ_12_24_units',
 'econ_12_30_units',
 'econ_16_6_units',
 'cider_12_6_units',
 'cider_12_12_units',
 'cider_12_24_units',
 'locraft_12_6_units',
 'natcraft_12_6_units',
 'topcraft_12_6_units',
 'natcraft_12_12_units',
 'locraft_12_12_units',
 'topcraft_12_12_units',
 'topcraft_16_12_units',
 'natcraft_12_15_units',
 'natcraft_12_18_units',
 'natcraft_12_24_units',
 'fmb_12_6_units',
 'fmb_12_12_units',
 'seltz_12_6_units',
 'fmb_12_8_units',
 'seltz_12_12_units',
 'seltz_12_24_units',
 'euro_12_6_units',
 'mex_12_6_units',
 'euro_12_12_units',
 'mex_12_12_units',
 'euro_12_18_units',
 'euro_12_24_units',
 'prem_12_6_units',
 'prem_12_12_units',
 'mex_12_15_units',
 'mex_12_18_units',


In [59]:
#Split data into independent and dependentvariables
X = etdata[['year_2018',
 'year_2019',
 'year_2020',
 'Apr',
 'Aug',
 'Dec',
 'Feb',
 'Jul',
 'Jun',
 'Mar',
 'May',
 'Nov',
 'Oct',
 'Sep',
 'log_dep_asrt',
 'log_ao_vol',
 'log_ao_price',
 'log_econ_12_12_price',
 'log_econ_12_15_price',
 'log_econ_12_18_price',
 'log_econ_12_24_price',
 'log_econ_12_30_price',
 'log_econ_16_6_price',
 'log_cider_12_6_price',
 'log_cider_12_12_price',
 'log_cider_12_24_price',
 'log_locraft_12_6_price',
 'log_natcraft_12_6_price',
 'log_topcraft_12_6_price',
 'log_natcraft_12_12_price',
 'log_locraft_12_12_price',
 'log_topcraft_12_12_price',
 'log_topcraft_16_12_price',
 'log_natcraft_12_15_price',
 'log_natcraft_12_18_price',
 'log_natcraft_12_24_price',
 'log_fmb_12_6_price',
 'log_fmb_12_12_price',
 'log_seltz_12_6_price',
 'log_fmb_12_8_price',
 'log_seltz_12_12_price',
 'log_seltz_12_24_price',
 'log_euro_12_6_price',
 'log_mex_12_6_price',
 'log_euro_12_12_price',
 'log_mex_12_12_price',
 'log_euro_12_18_price',
 'log_euro_12_24_price',
 'log_prem_12_6_price',
 'log_prem_12_12_price',
 'log_mex_12_15_price',
 'log_mex_12_18_price',
 'log_prem_12_18_price',
 'log_prem_12_20_price',
 'log_mex_12_24_price',
 'log_prem_12_30_price',
 'log_mex_12_30_price',
 'log_prem_12_36_price',
 'log_econ_12_6_price',
 'log_prem_12_24_price',
 'log_prem_16_8_price',
 'log_prem_16_12_price',
 'log_prem_16_20_price',
 'log_prem_16_6_price',
 'log_prem_16_9_price',
 'log_prem_16_15_price',
 'log_super_12_6_price',
 'log_super_12_8_price',
 'log_super_12_12_price',
 'log_super_12_18_price',
 'log_super_12_20_price',
 'log_super_12_24_price']].values
Y = modeling_df['log_dep_vol'].values

In [136]:
etdata = modeling_df.set_index(['tdoutletcd','weekendingdate'])
PanelOLS(etdata.log_dep_vol,etdata[['year_2018','year_2019','year_2020','log_dep_asrt', 
 'log_prem_12_24_price',
 'log_prem_12_20_price',
 'log_prem_16_8_price',
 'log_prem_12_6_price',
 'log_prem_12_12_price',
 'log_prem_12_18_price',
 'log_prem_12_30_price',
 'log_prem_16_12_price',
 'log_prem_16_20_price',
# 'log_prem_16_6_price',
 'log_prem_16_9_price', 
 'log_prem_12_36_price',
 'log_prem_16_15_price',
#  'log_econ_12_12_price',
#  'log_econ_12_15_price',
 'log_econ_12_18_price',
#  'log_econ_12_24_price',
#  'log_econ_12_30_price',
#  'log_econ_12_6_price',
 'log_econ_16_6_price',
 'log_euro_12_12_price',
#  'log_euro_12_18_price',
 'log_euro_12_24_price',
 'log_euro_12_6_price', 
 #'log_fmb_12_12_price',
 'log_fmb_12_6_price',
#  'log_fmb_12_8_price', 
 'log_locraft_12_12_price',
#  'log_locraft_12_6_price',
 'log_mex_12_12_price',
#  'log_mex_12_15_price',
 'log_mex_12_18_price',
 'log_mex_12_24_price',
 'log_mex_12_30_price',
#  'log_mex_12_6_price',
 'log_natcraft_12_12_price',
 'log_natcraft_12_15_price',
#  'log_natcraft_12_18_price',
#  'log_natcraft_12_24_price',
 'log_natcraft_12_6_price',
 'log_seltz_12_12_price',
#  'log_seltz_12_24_price',
#  'log_seltz_12_6_price',
#  'log_super_12_12_price',
#  'log_super_12_18_price',
#  'log_super_12_20_price',
#  'log_super_12_24_price',
 'log_super_12_6_price',
#  'log_super_12_8_price',
#  'log_topcraft_12_12_price',
#  'log_topcraft_12_6_price',
#  'log_topcraft_16_12_price',
'log_ao_vol','log_ao_price','Jan'
# ,'Feb'
,'Mar','Apr','Jun','Jul','Aug','Sep','Oct','Nov','Dec']],
         entity_effects=True).fit(debiased=True)

Dep. Variable:,log_dep_vol,R-squared:,0.6094
Estimator:,PanelOLS,R-squared (Between):,0.6910
No. Observations:,70278,R-squared (Within):,0.6094
Date:,"Fri, Aug 28 2020",R-squared (Overall):,0.6800
Time:,12:37:20,Log-likelihood,2.119e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2466.2
Entities:,684,P-value,0.0000
Avg Obs:,102.75,Distribution:,"F(44,69550)"
Min Obs:,18.000,,
Max Obs:,104.00,F-statistic (robust):,6.519e+05


In [88]:
modeling_df.year_2018 = modeling_df.year_2018.astype('float64')
pd.set_option('display.max_rows', 500)
print(modeling_df.dtypes)

tdoutletcd                          object
weekendingdate              datetime64[ns]
total_units                          int64
total_rev                          float64
total_vol                          float64
total_focus_units                    int64
total_focus_rev                    float64
total_focus_vol                    float64
dep_rev                            float64
dep_asrt                             int64
dep_units                            int64
dep_vol                            float64
ao_vol                             float64
ao_rev                             float64
ao_units                             int64
ao_price                           float64
econ_12_12_units                     int64
econ_12_15_units                     int64
econ_12_18_units                     int64
econ_12_24_units                     int64
econ_12_30_units                     int64
econ_16_6_units                      int64
cider_12_6_units                     int64
cider_12_12

In [ ]:
#coeff_df.to_csv('coeff_df.csv',index=False)

In [ ]:
regressor.score(X, y)